In [3]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [31]:
import pandas as pd
import numpy as np
import torch

import src.preprocessing as pre
import plotly.graph_objects as go
import src.utils as utils
from torch.utils.data import DataLoader



In [5]:
train, test, mappings, meta = pre.separate_year('../data/meta.json','../data/y.csv','../data/mappings.json')

In [58]:
production_plan = pre.decode(meta,mappings)[['Year','WeekTransplanted','Ranch','Class','Type','Variety','Ha']]

In [7]:
model = utils.create_model(train,num_epochs=100)

Epoch [1/200], Loss: 4382907.2223
Epoch [2/200], Loss: 3745032.2528
Epoch [3/200], Loss: 3165604.0142
Epoch [4/200], Loss: 3030348.3949
Epoch [5/200], Loss: 2858278.2528
Epoch [6/200], Loss: 2664212.1051
Epoch [7/200], Loss: 2501671.6406
Epoch [8/200], Loss: 2428333.4183
Epoch [9/200], Loss: 2380930.5994
Epoch [10/200], Loss: 2340200.2230
Epoch [11/200], Loss: 2299522.6428
Epoch [12/200], Loss: 2273494.2599
Epoch [13/200], Loss: 2257141.0781
Epoch [14/200], Loss: 2210988.5881
Epoch [15/200], Loss: 2182991.4922
Epoch [16/200], Loss: 2151938.9538
Epoch [17/200], Loss: 2112385.1040
Epoch [18/200], Loss: 2080347.7195
Epoch [19/200], Loss: 2044036.1896
Epoch [20/200], Loss: 2005300.2287
Epoch [21/200], Loss: 1981089.7152
Epoch [22/200], Loss: 1924149.6168
Epoch [23/200], Loss: 1883714.1349
Epoch [24/200], Loss: 1850999.5515
Epoch [25/200], Loss: 1805585.1445
Epoch [26/200], Loss: 1765487.3288
Epoch [27/200], Loss: 1735544.3714
Epoch [28/200], Loss: 1694341.1058
Epoch [29/200], Loss: 1666078

In [60]:
preds = utils.predict_harvest(model,test)

In [61]:
production_plan['PredictedTotalKilos'] = preds.sum(axis=1)
production_plan['ActualTotalKilos'] = test.Y.detach().numpy().sum(axis=1)

In [41]:
summary = production_plan.groupby(['Class','Type']).agg({'Ha':'sum','PredictedTotalKilos':'sum','ActualTotalKilos':'sum'})

In [44]:
def predict_harvest(_model, _test_dataset):
    _model.eval()
    with torch.no_grad():
        test_predictions = np.zeros((len(_test_dataset), 15,20))
        
    
        test_loader = DataLoader(_test_dataset, batch_size=64, shuffle=False)
        for batch in test_loader:
            features, ranch_id, class_id, type_id, variety_id, climate_data, y, bounds, idx = batch
            batch_size = y.size(0)
            log_kilos = torch.log1p(y) 
            week_numbers = torch.arange(0, 20).unsqueeze(0).repeat(batch_size,1)
            inputs = torch.stack([y, log_kilos, week_numbers], dim=2)
            for i in range(5,20):
                kilo_inputs = inputs[:,:i,:]
                outputs = _model(features, ranch_id, class_id, type_id, variety_id, climate_data, kilo_inputs)
                test_predictions[idx,i-5,:] = outputs.detach().numpy()

    return test_predictions

In [62]:
preds = predict_harvest(model,test)

In [64]:
actuals = test.Y.detach().numpy()

In [65]:
preds[:,0,:]

array([[-1.76117551e+00, -2.31934190e-01, -2.49751471e-02, ...,
         3.82754492e+03,  3.38743896e+03,  2.41216211e+03],
       [-1.78795481e+00, -3.30950052e-01, -1.08312540e-01, ...,
         2.47356885e+03,  2.14371655e+03,  1.48200110e+03],
       [-1.61818945e+00, -2.27434456e-01, -8.10581371e-02, ...,
         2.37995557e+03,  2.07076807e+03,  1.43295239e+03],
       ...,
       [-1.14866352e+00, -2.53558904e-01,  2.14779526e-02, ...,
         1.45277954e+03,  1.23839197e+03,  8.76833740e+02],
       [-1.01850522e+00, -8.25120509e-02,  2.10033208e-02, ...,
         6.27791443e+02,  5.06521423e+02,  3.21959290e+02],
       [-1.28657043e+00, -2.28704050e-01, -2.89215967e-02, ...,
         1.39987488e+03,  1.17342749e+03,  8.17523499e+02]])